## Yasmine Dawud
## April 16, 2023
## CS379-2302A-01
## Individual Project 1 - Unsupervised Machine Learning

In [79]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [80]:
#read CSV
titanic_df = pd.read_csv("/kaggle/input/titanic-dataset-csv/CS379T-Week-1-IP.csv")
titanic_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [81]:
# check missing values in data
titanic_df.isnull().sum()

pclass          1
survived        1
name            1
sex             1
age           264
sibsp           1
parch           1
ticket          1
fare            2
cabin        1015
embarked        3
boat          824
body         1189
home.dest     565
dtype: int64

In [82]:
titanic_df.drop(['body','name'], 1, inplace=True)
titanic_df.fillna(0, inplace=True)
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1310 non-null   float64
 1   survived   1310 non-null   float64
 2   sex        1310 non-null   object 
 3   age        1310 non-null   float64
 4   sibsp      1310 non-null   float64
 5   parch      1310 non-null   float64
 6   ticket     1310 non-null   object 
 7   fare       1310 non-null   float64
 8   cabin      1310 non-null   object 
 9   embarked   1310 non-null   object 
 10  boat       1310 non-null   object 
 11  home.dest  1310 non-null   object 
dtypes: float64(6), object(6)
memory usage: 122.9+ KB


In [83]:
def handle_non_numerical_data(titanic_df):
    columns = titanic_df.columns.values
    for column in columns:
        text_digit_vals = {}
        #ex. {'Female': 0, 'Male': 1}
        def convert_to_int(val):
            return text_digit_vals[val]
        #this is asking if the column is numerical. If not, it will populate the dict above
        if titanic_df[column].dtype != np.int64 and titanic_df[column].dtype != np.float64:
            column_contents = titanic_df[column].values.tolist()
            unique_elements = set(column_contents) #This will give us all unique non-repetitive values
            x = 0
            #if not numerical, converts to list, gets the set, populates the dict with the unique elements and changes to ints
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            titanic_df[column] = list(map(convert_to_int, titanic_df[column]))
        
    return titanic_df

In [84]:
titanic_df = handle_non_numerical_data(titanic_df)

titanic_df.drop(['cabin','sibsp', 'embarked', 'home.dest'],1,inplace=True) #came back to do this for better accuracy

print(titanic_df.head())


   pclass  survived  sex      age  parch  ticket      fare  boat
0     1.0       1.0    2  29.0000    0.0     391  211.3375    12
1     1.0       1.0    1   0.9167    2.0     198  151.5500    11
2     1.0       0.0    2   2.0000    2.0     198  151.5500     0
3     1.0       0.0    1  30.0000    2.0     198  151.5500     0
4     1.0       0.0    2  25.0000    2.0     198  151.5500     0


In [85]:
X = np.array(titanic_df.drop(['survived'], 1).astype(float))
X= preprocessing.scale(X)
y=np.array(titanic_df['survived'])

clf = KMeans(n_clusters=2)
clf.fit(X)

KMeans(n_clusters=2)

In [86]:
titanic_df['predicted'] = clf.predict(X)

print(titanic_df[titanic_df['predicted'] ==0]['survived'].describe())

print(titanic_df[titanic_df['predicted'] ==1]['survived'].describe())


count    907.000000
mean       0.239250
std        0.426861
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: survived, dtype: float64
count    403.000000
mean       0.702233
std        0.457845
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: survived, dtype: float64


In [87]:
#dropping cabin data column due to missing data
def preProcess(titanic_df):
    deckType = {"A": 1, "B": 2, "C": 3, "D":4, "E": 5, "F":6, "G": 7, "U":8}
    titanic_df['Cabin'] = titanic_df['Cabin'].fillna("U0")
    titnaic_df['Deck'] = titanic_df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    titanic_df['Deck'] = titanic_df['Deck'].map(deckType)
    titanic_df['Deck'] = titanic_df['Deck'].fillna(0)
    titanic_df['Deck'] = titanic_df['Deck'].astype(int)
    
    titanic_df = titanic_df.drop(['Cabin'], axis=1)

In [88]:
#Handling age data column missing values
mean = titanic_df["age"].mean()
std = titanic_df["age"].std()
is_null = titanic_df["age"].isnull().sum()

In [89]:
#Compute random numbers between the mean,std and is_null
try:
    rand_age = np.random.radiant(mean - std, mean + std, size = is_null)
except:
    rand_age = 0

In [90]:
#Fill NaN values in age column with random values generated
age_slice = titanic_df["age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
titanic_df["age"] = age_slice
titanic_df["age"] = titanic_df["age"].astype(int)

In [91]:
#converting fare from float to int
titanic_df['fare'] = titanic_df['fare'].fillna(0)
titanic_df['fare'] = titanic_df['fare'].astype(int)

In [92]:
#converting genders into 0 or 1
gender = {"male":0, "female":1}
titanic_df['sex'] = titanic_df['sex'].map(gender)

In [93]:
#dropping ticket from dataset as data is not useful
titanic_df = titanic_df.drop(['ticket'], axis=1)

In [94]:
#creating age group
titanic_df['age'] = titanic_df['age'].astype(int)
titanic_df.loc[titanic_df['age'] <= 11, 'age'] = 0
titanic_df.loc[(titanic_df['age'] > 11) & (titanic_df['age'] <= 18), 'age'] = 1
titanic_df.loc[(titanic_df['age'] > 18) & (titanic_df['age'] <= 22), 'age'] = 2
titanic_df.loc[(titanic_df['age'] > 22) & (titanic_df['age'] <= 27), 'age'] = 3
titanic_df.loc[(titanic_df['age'] > 27) & (titanic_df['age'] <= 33), 'age'] = 4
titanic_df.loc[(titanic_df['age'] > 33) & (titanic_df['age'] <= 40), 'age'] = 5
titanic_df.loc[(titanic_df['age'] > 40) & (titanic_df['age'] <= 66), 'age'] = 6
titanic_df.loc[titanic_df['age'] > 66, 'age'] = 6

In [95]:
# Creating Fare group
titanic_df.loc[titanic_df['fare'] <= 7.91, 'fare'] = 0
titanic_df.loc[(titanic_df['fare'] > 7.91) & (titanic_df['fare'] <= 14.454), 'fare'] = 1
titanic_df.loc[(titanic_df['fare'] > 14.454) & (titanic_df['fare'] <= 31), 'fare']   = 2
titanic_df.loc[(titanic_df['fare'] > 31) & (titanic_df['fare'] <= 99), 'fare']   = 3
titanic_df.loc[(titanic_df['fare'] > 99) & (titanic_df['fare'] <= 250), 'fare']   = 4
titanic_df.loc[titanic_df['fare'] > 250, 'fare'] = 5
titanic_df['fare'] = titanic_df['fare'].astype(int)

In [96]:
# Adding new column for Age Class
titanic_df['age_class']= titanic_df['age']* titanic_df['pclass']

In [97]:
# Silencing Future Warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [98]:
def sType(str):
  if (str == '1'):
    return "Survived"
  else:
    return "Deceased"

In [99]:
def printPrediction(predList):
  i = 1
  s = ""
  for cell in predList:
    s = s + (str(i) + ": " +sType(str(cell))+ '\n')
    i = i + 1
  
  return s

In [100]:
global predLog 

In [103]:
# Logistic Regression Machine Learning Model
def logRegression(train1,train2,test):
  model = LogisticRegression()
  model.fit(train1, train2)
  prediction = model.predict(test)
  global predLog
  predLog = prediction
  accuracy = round(model.score(train1,train2)*100,2)
  return 'Predicting... \n \n'+ printPrediction(prediction) + '\nModel Accuracy: ' + str(accuracy)+'%'

In [102]:
def printPrediction(predList):
  i = 1
  s = ""
  for cell in predList:
    s = s + (str(i) + ": " +sType(str(cell))+ '\n')
    i = i + 1
  
  return s